In [55]:
# import tensorflow as tf
import random
import numpy as np
from vizdoom import *
from collections import deque
from skimage import transform

# Setup environment 

In [11]:
def create_env():
    game = DoomGame()
    game.load_config("./ViZDoom-1.1.5pre-Win-Python36-x86_64/ViZDoom-1.1.5pre-Win-Python36-x86_64/vizdoom/scenarios/basic.cfg")
    game.set_doom_scenario_path("./ViZDoom-1.1.5pre-Win-Python36-x86_64/ViZDoom-1.1.5pre-Win-Python36-x86_64/vizdoom/scenarios/basic.wad")
#     game.init()
    
    shoot = [0,0,1]
    left = [1,0,0]
    right = [0,1,0]
    possible_action = [shoot,left,right]
    
    return game,possible

In [9]:
def test():
    game = DoomGame()
    game.load_config("./ViZDoom-1.1.5pre-Win-Python36-x86_64/ViZDoom-1.1.5pre-Win-Python36-x86_64/vizdoom/scenarios/basic.cfg")
    game.set_doom_scenario_path("./ViZDoom-1.1.5pre-Win-Python36-x86_64/ViZDoom-1.1.5pre-Win-Python36-x86_64/vizdoom/scenarios/basic.wad")
#     game.set_mode(Mode.ASYNC_PLAYER)

    game.init()

    shoot = [0,0,1]
    left = [1,0,0]
    right = [0,1,0]

    episodes = 10
    for i in range(episodes):
        game.new_episode()
        while not game.is_episode_finished():
            state = game.get_state()
            img = state.screen_buffer
            misc = state.game_variables
            action = random.choice([shoot, left, right]) # A random agent
            reward = game.make_action(action)
            time.sleep(0.02) # Just to make it slower
            
            
        print("Total reward: " + str(game.get_total_reward()))
    game.close()

In [40]:
class inp_data:

    def __init__(self):
        self.que = deque(maxlen=4)
    
    def add(self,frame):
        self.que.append(frame)
        return np.stack(self.que,axis=0)

In [56]:
def preprocess(frame):
    
    frame = frame.mean(axis=0,dtype = np.int)
    crop_frame = frame[30:-10,30:-30]
    frame = crop_frame/255
    return transform.resize(frame,[84,84])